# van der Waals equation of state

use SymPy to find expressions for substance-specific constants, by taking these derivatives and applying the constraints at the critical point:

In [1]:
import sympy
sympy.init_printing(use_latex='mathjax')

R, Tc, vc, Pc, a, b = sympy.symbols('R, T_c, v_c, P_c, a, b', real=True)

equation_state = (R*Tc)/(vc - b) - (a/vc**2)

# first derivative with respect to v
first_deriv = sympy.diff(equation_state, vc)
display(sympy.Eq(first_deriv, 0))

second_deriv = sympy.diff(first_deriv, vc)
display(sympy.Eq(second_deriv, 0))

     R⋅T_c      2⋅a     
- ─────────── + ──── = 0
            2      3    
  (-b + v_c)    v_c     

  2⋅R⋅T_c     6⋅a     
─────────── - ──── = 0
          3      4    
(-b + v_c)    v_c     

solve this system of two equations for the two unknowns(a and b):

In [2]:
sol = sympy.solve((
    first_deriv, 
    second_deriv, 
    ), (a, b), dict=True)

display(sol[0])

a_expr = sol[0][a]
b_expr = sol[0][b]

⎧   9⋅R⋅T_c⋅v_c     v_c⎫
⎨a: ───────────, b: ───⎬
⎩        8           3 ⎭

This gives us the coefficients a and b as a function of T crit, P crit, and R. However, while the critical temperature can be measured experimentally relatively easily, the critical specific volume is harder to measure accurately.

So, we can introduce the equation of state applied at the critical point,and then solve the system of three equations for a, b, and v crit to eliminate the critical specific volume:

In [3]:
sol = sympy.solve((
    a - a_expr, b - b_expr, Pc - ((R*Tc / (vc-b)) - (a / vc**2))
    ), (a, b, vc), dict=True)
display(sol[0])

⎧       2    2                        ⎫
⎪   27⋅R ⋅T_c      R⋅T_c       3⋅R⋅T_c⎪
⎨a: ──────────, b: ─────, v_c: ───────⎬
⎪     64⋅P_c       8⋅P_c        8⋅P_c ⎪
⎩                                     ⎭

Thus, we can now determine the coefficients a and b based on measured values of the critical temperature and pressure.
It turns out that the van der Waals equation of state gives a cubic function of v when expressed in terms of reduced properties.

This means that we need to find the roots of this cubic polynomial to get specific volume. As a consequence, there will be three roots and potential solutions for vr. At minimum, we should only consider real roots; imaginary roots are not physically meaningful.

First, consider when Tr>1.0, such as for Tr = 2.5 and Pr = 2.0:

In [5]:
import numpy as np

temp_r = 2.5
pres_r = 2.0
coeffs = [
    1.0, 
    -((8.0*temp_r / (3.0*pres_r)) + (1.0 / 3.0)),
    3.0 / pres_r,
    -1.0 / pres_r
    ]
roots = np.roots(coeffs)
print(f'Roots: {roots}')
# get only real roots
roots = roots.real[np.abs(roots.imag)<1e-5]

print(f'Number of real roots: {len(roots)}')

print(f'T_r = {temp_r: .2f}')
print(f'P_r = {pres_r: .2f}')

reduced_volumes = ','.join([f'{vol_r: .3f}' for vol_r in roots])
print(f'v_r = {reduced_volumes}')

Roots: [3.25277894+0.j         0.20694386+0.33299993j 0.20694386-0.33299993j]
Number of real roots: 1
T_r =  2.50
P_r =  2.00
v_r =  3.253


So, for temperatures above the critical temperature, there will be only one real value for reduced specific volume.

But, if Tr < 1.0, we can get three real roots:

In [6]:
temp_r = 0.90
pres_r = 0.61
coeffs = [
    1.0, 
    -((8.0*temp_r / (3.0*pres_r)) + (1.0 / 3.0)),
    3.0 / pres_r,
    -1.0 / pres_r
    ]
roots = np.roots(coeffs)
roots = roots.real[np.abs(roots.imag)<1e-5]

print(f'Number of real roots: {len(roots)}')

print(f'T_r = {temp_r: .2f}')
print(f'P_r = {pres_r: .2f}')

reduced_volumes = ','.join([f'{vol_r: .3f}' for vol_r in roots])
print(f'v_r = {reduced_volumes}')

Number of real roots: 3
T_r =  0.90
P_r =  0.61
v_r =  2.640, 1.017, 0.610


In this case, we need to apply some additional knowledge to understand these roots and identify any non-physical values